In [ ]:
import requests
from bs4 import BeautifulSoup
import scipy.io

import pandas as pd
import numpy as np

from time import sleep
import timeit

In [ ]:
r = requests.get("http://www.boardgamegeek.com/xmlapi2/user?name=Zazz&top=1")
soup = BeautifulSoup(r.text, "html.parser")  # Use the xml parser for API responses and the html_parser for scraping
print(r.status_code)  # 404 not found and the like. Hopefully 200!

200


In [ ]:
from google.colab import drive
import os

drive.mount('/content/gdrive')
#path of your file
os.chdir('/content/gdrive/My Drive/Pandas/')  

Mounted at /content/gdrive


In [ ]:
url = r"https://boardgamegeek.com/boardgame/174430/gloomhaven"
page_data = requests.get(url)
soup = BeautifulSoup(page_data.content, 'html.parser')

# Don't run this because the rank change daily

In [ ]:
## scrape from rank page to get url ##

# Initialize a DF to hold all our scraped game info
df_all = pd.DataFrame(columns=["id", "name", "rank", "url"])
max_rank = 1
npage = 1
rank = 0
# Scrap successful pages in the results until we get down to games with < 1000 ratings each
while max_rank < 2001 and npage < 21 :
    # Get full HTML for a specific page in the full listing of boardgames sorted by 
    r = request("https://boardgamegeek.com/browse/boardgame/page/%i?sort=rank" % (npage,))
    soup = BeautifulSoup(r.text, "html.parser")    
    
    # Get rows for the table listing all the games on this page
    table = soup.find_all("tr", attrs={"id": "row_"})  # Get list of all the rows (tags) in the list of games on this page
    df = pd.DataFrame(columns=["id", "name", "rank", "url"], index=range(len(table)))  # DF to hold this pages results
    
    # Loop through each row and pull out the info for that game
    for idx, row in enumerate(table):
        # Row may or may not start with a "boardgame rank" link, if YES then strip it
        links = row.find_all("a")
        if "name" in links[0].attrs.keys():
            del links[0]
        gamelink = links[1]  # Get the relative URL for the specific game
        gameid = int(gamelink["href"].split("/")[2])  # Get the game ID by parsing the relative URL
        gamename = gamelink.contents[0]  # Get the actual name of the game as the link contents
        gameshortlink = gamelink["href"]
        rank = rank+1

        df.iloc[idx, :] = [gameid, gamename, rank, gameshortlink]

    print("Page %i scraped" % (npage))
    df_all = pd.concat([df_all, df], axis=0)
    npage += 1
    sleep(2) # Keep the BGG server happy.

Page 1 scraped
Page 2 scraped
Page 3 scraped
Page 4 scraped
Page 5 scraped
Page 6 scraped
Page 7 scraped
Page 8 scraped
Page 9 scraped
Page 10 scraped
Page 11 scraped
Page 12 scraped
Page 13 scraped
Page 14 scraped
Page 15 scraped
Page 16 scraped
Page 17 scraped
Page 18 scraped
Page 19 scraped
Page 20 scraped


In [ ]:
df = df_all.copy()
# Reset the index since we concatenated a bunch of DFs with the same index into one DF
df.reset_index(inplace=True, drop=True)
# Write the DF to .csv for future use
df.head()

,id,name,rank,url
0,174430,Gloomhaven,1,/boardgame/174430/gloomhaven
1,224517,Brass: Birmingham,2,/boardgame/224517/brass-birmingham
2,161936,Pandemic Legacy: Season 1,3,/boardgame/161936/pandemic-legacy-season-1
3,342942,Ark Nova,4,/boardgame/342942/ark-nova
4,233078,Twilight Imperium: Fourth Edition,5,/boardgame/233078/twilight-imperium-fourth-edi...


In [ ]:
df['rank']

0          1
1          2
2          3
3          4
4          5
        ... 
1995    1996
1996    1997
1997    1998
1998    1999
1999    2000
Name: rank, Length: 2000, dtype: object

In [ ]:
df.to_csv("bgg_top2000.csv", index=False, encoding="utf-8")

End of scraping

##Next step

In [ ]:
df = pd.read_csv("bgg_top2000.csv")

In [ ]:
#url part 1
up1 = r"https://boardgamegeek.com"
#URLs from the data frame
urls = df['url']
#index variable
n = 0
#list of descriptions
desc = []
#for loop
for u in urls:
    url = up1 + urls[n]
    print(url)
    d = requests.get(url)
    soup = BeautifulSoup(d.content, 'html.parser')
    s = soup.contents[2].get_text
    s = str(s)
    index_script = s.find('<script>')
    s = s[index_script:]
    index_desc = s.find('"description"')
    index_enddesc = s.find('"wiki"')
    s_desc = s[index_desc+14:index_enddesc-1]
    (desc.append(s_desc) if s_desc else desc.append("No Description"))
    n+=1

https://boardgamegeek.com/boardgame/174430/gloomhaven
https://boardgamegeek.com/boardgame/224517/brass-birmingham
https://boardgamegeek.com/boardgame/161936/pandemic-legacy-season-1
https://boardgamegeek.com/boardgame/342942/ark-nova
https://boardgamegeek.com/boardgame/233078/twilight-imperium-fourth-edition
https://boardgamegeek.com/boardgame/167791/terraforming-mars
https://boardgamegeek.com/boardgame/291457/gloomhaven-jaws-lion
https://boardgamegeek.com/boardgame/187645/star-wars-rebellion
https://boardgamegeek.com/boardgame/115746/war-ring-second-edition
https://boardgamegeek.com/boardgame/162886/spirit-island
https://boardgamegeek.com/boardgame/220308/gaia-project
https://boardgamegeek.com/boardgame/316554/dune-imperium
https://boardgamegeek.com/boardgame/182028/through-ages-new-story-civilization
https://boardgamegeek.com/boardgame/12333/twilight-struggle
https://boardgamegeek.com/boardgame/193738/great-western-trail
https://boardgamegeek.com/boardgame/169786/scythe
https://board

In [ ]:
#url part 1
up1 = r"https://boardgamegeek.com"
#URLs from the data frame
urls = df['url']
#index variable
n = 0
#list of descriptions
desc = []
#for loop
for u in urls:
    url = up1 + urls[n]
    print(url)
    d = requests.get(url)
    soup = BeautifulSoup(d.content, 'html.parser')
    s = soup.contents[2].get_text
    s = str(s)
    index_script = s.find('<script>')
    s = s[index_script:]
    index_desc = s.find('"description"')
    index_enddesc = s.find('"wiki"')
    s_desc = s[index_desc+14:index_enddesc]
    (desc.append(s_desc) if s_desc else desc.append("No Description"))
    n+=1

https://boardgamegeek.com/boardgame/174430/gloomhaven
https://boardgamegeek.com/boardgame/224517/brass-birmingham
https://boardgamegeek.com/boardgame/161936/pandemic-legacy-season-1
https://boardgamegeek.com/boardgame/342942/ark-nova
https://boardgamegeek.com/boardgame/233078/twilight-imperium-fourth-edition
https://boardgamegeek.com/boardgame/167791/terraforming-mars
https://boardgamegeek.com/boardgame/291457/gloomhaven-jaws-lion
https://boardgamegeek.com/boardgame/187645/star-wars-rebellion
https://boardgamegeek.com/boardgame/115746/war-ring-second-edition
https://boardgamegeek.com/boardgame/162886/spirit-island
https://boardgamegeek.com/boardgame/220308/gaia-project
https://boardgamegeek.com/boardgame/316554/dune-imperium
https://boardgamegeek.com/boardgame/182028/through-ages-new-story-civilization
https://boardgamegeek.com/boardgame/12333/twilight-struggle
https://boardgamegeek.com/boardgame/193738/great-western-trail
https://boardgamegeek.com/boardgame/169786/scythe
https://board

In [ ]:
df['description'] = desc

In [ ]:
df

,Unnamed: 0,id,name,rank,url,description
0,0,174430,Gloomhaven,1,/boardgame/174430/gloomhaven,"""<p><em><strong>Gloomhaven <\/strong><\/em> is..."
1,1,224517,Brass: Birmingham,2,/boardgame/224517/brass-birmingham,"""<p><em><strong>Brass: Birmingham<\/strong><\/..."
2,2,161936,Pandemic Legacy: Season 1,3,/boardgame/161936/pandemic-legacy-season-1,"""<p><em><strong>Pandemic Legacy<\/strong><\/em..."
3,3,342942,Ark Nova,4,/boardgame/342942/ark-nova,"""<p>In <em><strong>Ark Nova<\/strong><\/em>, y..."
4,4,233078,Twilight Imperium: Fourth Edition,5,/boardgame/233078/twilight-imperium-fourth-edi...,"""<p><em><strong>Twilight Imperium (Fourth Edit..."
...,...,...,...,...,...,...
1995,1995,3720,Subbuteo,1996,/boardgame/3720/subbuteo,"""<p><strong>Subbuteo<\/strong> is a classic fi..."
1996,1996,227072,The Chameleon,1997,/boardgame/227072/chameleon,"""<p>A bluffing deduction game for everyone.<\/..."
1997,1997,192802,Days of Ire: Budapest 1956,1998,/boardgame/192802/days-ire-budapest-1956,"""<p><em>It's 1956, and waves of protest in Pol..."
1998,1998,209001,Monster Lands,1999,/boardgame/209001/monster-lands,"""<p><em><strong>Monster Lands<\/strong><\/em> ..."


In [ ]:
df.to_excel('bgg_top2000longdesc.xlsx')
df.to_csv('bgg_top2000longdesc.csv')

## Scrape other stat

In [ ]:
df = pd.read_csv('bgg_top2000.csv')
df.head()

,Unnamed: 0,id,name,rank,url
0,0,174430,Gloomhaven,1,/boardgame/174430/gloomhaven
1,1,224517,Brass: Birmingham,2,/boardgame/224517/brass-birmingham
2,2,161936,Pandemic Legacy: Season 1,3,/boardgame/161936/pandemic-legacy-season-1
3,3,342942,Ark Nova,4,/boardgame/342942/ark-nova
4,4,233078,Twilight Imperium: Fourth Edition,5,/boardgame/233078/twilight-imperium-fourth-edi...


In [ ]:
# desc , stats , categories and mechanics scraped

#url part 1
up1 = r"https://boardgamegeek.com"
#URLs from the data frame
urls = df['url']
#index variable
n = 0
#list of descriptions
desc = []
minp = []
maxp = []
age = []
w = []
rating = []
numown = []
numplay = []
y = []
mintime = []
maxtime = []
minimumage = []
category = []
mechanics = []
#for loop
for u in urls:
    url = up1 + urls[n]
    print(url)
    d = requests.get(url)
    soup = BeautifulSoup(d.content, 'html.parser')
    s = soup.contents[2].get_text
    s = str(s)
    index_script = s.find('<script>')
    s = s[index_script:]
    index_desc = s.find('"description"')
    index_enddesc = s.find('"wiki"')
    s_desc = s[index_desc+14:index_enddesc-1]
    (desc.append(s_desc) if s_desc else desc.append("No Description"))


    s = soup.contents[2].get_text
    s = str(s)
    index = s.find('{"title":"Board Game Forums"')
    s = s[index:]
    index_userplayers = s.find('"userplayers"')
    index_stats = s.find('"stats"')
    polls = s[index_userplayers-1:index_stats-1]
    if "null" in polls :
      polls = polls.replace("null", '"null"')
    polls = eval(polls)
    bestmin = polls['userplayers']['best'][0]['min']
    bestmax = polls['userplayers']['best'][0]['max']
    playerage = polls["playerage"]
    weight = polls["boardgameweight"]["averageweight"]

    s = s[index_userplayers:]
    index_relatedcounts = s.find('"relatedcounts"')
    index_stats = s.find('"stats"')
    stats = s[index_stats+8:index_relatedcounts-1] 
    stats = eval(stats)
    average_rating = stats["average"]
    numowned = stats["numowned"]
    numplayed = stats["numplays"]
    index_basicinfo = s.find('"yearpublished"')
    index_override_rankable = s.find('"override_rankable"')
    basicinfo = '{'+s[index_basicinfo:index_override_rankable-1]+'}'
    basicinfo = eval(basicinfo)
    year = basicinfo["yearpublished"]
    minplayer = basicinfo["minplayers"]
    maxplayer = basicinfo["maxplayers"]
    minplaytime = basicinfo["minplaytime"]
    maxplaytime = basicinfo["maxplaytime"]
    minage = basicinfo["minage"]


    index_cat = s.find('"boardgamecategory"')
    s = s[index_cat:]
    index_mech = s.find('"boardgamemechanic"')
    cat = s[0:index_mech-1]
    cat = '{'+cat+'}'
    if "\\" in cat :
      cat = cat.replace("\\", "")
    number_of_cat = cat.count('"name"')
    cat = eval(cat)
    catlist = []
    for i in range(0, number_of_cat) :
      a = [cat['boardgamecategory'][i]['name']]
      catlist = catlist + a

    index_mech = s.find('"boardgamemechanic"')
    s = s[index_mech:]
    index_exp = s.find('"boardgameexpansion"')
    mech = s[0:index_exp-1]
    mech = '{'+mech+'}'
    if "\\" in mech :
      mech = mech.replace("\\", "")
    number_of_mechanic = mech.count('"name"')
    mech = eval(mech)
    mechlist = []
    for i in range(0, number_of_mechanic) :
      a = [mech['boardgamemechanic'][i]['name']]
      mechlist = mechlist + a

    (minp.append(minplayer) if minplayer else minp.append("No Minimum"))
    (maxp.append(maxplayer) if maxplayer else maxp.append("No Maximum"))
    (age.append(playerage) if playerage else age.append("Not found"))
    (w.append(weight) if weight else w.append("Not found"))
    (rating.append(average_rating) if average_rating else rating.append("Not found"))
    (numown.append(numowned) if numowned else numown.append("Not found"))
    (numplay.append(numplayed) if numplayed else numplay.append("Not found"))
    (y.append(year) if year else year.append("Not found"))
    (mintime.append(minplaytime) if minplaytime else mintime.append("Not found"))
    (maxtime.append(maxplaytime) if maxplaytime else maxtime.append("Not found"))
    (minimumage.append(minage) if minage else minimumage.append("Not found"))
    (category.append(catlist) if catlist else category.append("Not found"))
    (mechanics.append(mechlist) if mechlist else mechanics.append("Not found"))

    n+=1

https://boardgamegeek.com/boardgame/174430/gloomhaven
https://boardgamegeek.com/boardgame/224517/brass-birmingham
https://boardgamegeek.com/boardgame/161936/pandemic-legacy-season-1
https://boardgamegeek.com/boardgame/342942/ark-nova
https://boardgamegeek.com/boardgame/233078/twilight-imperium-fourth-edition
https://boardgamegeek.com/boardgame/167791/terraforming-mars
https://boardgamegeek.com/boardgame/291457/gloomhaven-jaws-lion
https://boardgamegeek.com/boardgame/187645/star-wars-rebellion
https://boardgamegeek.com/boardgame/115746/war-ring-second-edition
https://boardgamegeek.com/boardgame/162886/spirit-island
https://boardgamegeek.com/boardgame/220308/gaia-project
https://boardgamegeek.com/boardgame/316554/dune-imperium
https://boardgamegeek.com/boardgame/182028/through-ages-new-story-civilization
https://boardgamegeek.com/boardgame/12333/twilight-struggle
https://boardgamegeek.com/boardgame/193738/great-western-trail
https://boardgamegeek.com/boardgame/169786/scythe
https://board

In [ ]:
#add the descriptions to the dataframe

df['year'] = y
df['weight'] = w
df['rating'] = rating
df['player age'] = age
df['min player'] = minp
df['max player'] = maxp
df['min playtime'] = mintime
df['max playtime'] = maxtime
df['Own'] = numown
df['all time plays'] = numplay
df['description'] = desc

In [ ]:
df['categories'] = category
df['mechanics'] = mechanics
df['year'] = y
df = df[['id', 'name', 'year', 'rank', 'weight', 'rating', 'player age', 'min player', 'max player', 'min playtime', 'max playtime', 'Own', 'all time plays', 'categories','mechanics', 'url', 'description']]
df.head()

,id,name,year,rank,weight,rating,player age,min player,max player,min playtime,max playtime,Own,all time plays,categories,mechanics,url,description
0,174430,Gloomhaven,2017,1,3.888937,8.66608,14+,1,4,60,120,88756,460866,"[Adventure, Exploration, Fantasy, Fighting, Mi...","[Action Queue, Action Retrieval, Campaign / Ba...",/boardgame/174430/gloomhaven,"""<p><em><strong>Gloomhaven <\/strong><\/em> is..."
1,224517,Brass: Birmingham,2018,2,3.898923,8.64654,14+,2,4,60,120,49975,90077,"[Economic, Industry / Manufacturing, Post-Napo...","[Hand Management, Income, Loans, Market, Netwo...",/boardgame/224517/brass-birmingham,"""<p><em><strong>Brass: Birmingham<\/strong><\/..."
2,161936,Pandemic Legacy: Season 1,2015,3,2.832344,8.56431,12+,2,4,60,60,76981,265929,"[Environmental, Medical]","[Action Points, Cooperative Game, Hand Managem...",/boardgame/161936/pandemic-legacy-season-1,"""<p><em><strong>Pandemic Legacy<\/strong><\/em..."
3,342942,Ark Nova,2021,4,3.714385,8.55794,12+,1,4,90,150,40256,115867,"[Animals, Economic, Environmental]","[End Game Bonuses, Hand Management, Hexagon Gr...",/boardgame/342942/ark-nova,"""<p>In <em><strong>Ark Nova<\/strong><\/em>, y..."
4,233078,Twilight Imperium: Fourth Edition,2017,5,4.295896,8.63909,14+,3,6,240,480,24776,31841,"[Civilization, Economic, Exploration, Negotiat...","[Action Drafting, Area Majority / Influence, A...",/boardgame/233078/twilight-imperium-fourth-edi...,"""<p><em><strong>Twilight Imperium (Fourth Edit..."


In [ ]:
df.to_excel('bgg_scraped.xlsx')
df.to_csv('bgg_scraped.csv')